# Introducción a Spark

## Primeros pasos

In [0]:
# Accedemos directamente a la instancia de SparkSession a través de la variable spark 

sc = spark.sparkContext

In [0]:
# Mostramos el contenido del contexto 

sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

#### Parallelize

In [0]:
# Creación de un RDD de ejemplo con los números del 1 al 100

rdd = sc.parallelize(range(100 +1))
suma = rdd.sum()
print(suma)

In [0]:
# Creación del RDD a partir de una lista

miRDD = sc.parallelize([1,2,3,4,5,6,7,8,9])

lista  = ['Hola', 'Adiós', 'Hasta luego']
listaRDD = sc.parallelize(lista) 

# Creamos otro RDD con 4 particiones
listaRDD4 = sc.parallelize(lista,4) 

#### Take y Sample

In [0]:
# usamos las listas anteriores

miRDD.take(3)   

Out[9]: [1, 2, 3]

In [0]:
listaRDD.take(2)

Out[10]: ['Hola', 'Adiós']

In [0]:
miRDDmuestra = miRDD.sample(False, 0.5)
miRDDmuestra.collect()

Out[11]: [2, 5, 6, 7]

In [0]:
# Obtener una muestra mediante takeSample que funciona de forma similar pero sin barajar
# El primer parámetro vuelve a indicar si hay repetidos, pero el segundo fija la cantidad de elementos a devolver

miRDDmuestraT = miRDD.takeSample(False, 5)
print(miRDDmuestraT)

[2, 8, 6, 3, 9]


In [0]:
# mediante top obtenemos los primeros elementos del RDD ordenado 
miRDD.top(3)

Out[13]: [9, 8, 7]

In [0]:
#  takeOrdered obtiene los registros ordenados ascendentemente (al contrario que top), con la opción de ordenarlos descendentemente 

miRDD.takeOrdered(3)    
miRDD.takeOrdered(3, lambda x: -x)

Out[14]: [9, 8, 7]

## Particiones

In [0]:
# Paralelizamos una lista y vemos el número de particiones utilizadas
rdd = sc.parallelize([1,1,2,2,3,3,4,5])
rdd.getNumPartitions() 

Out[1]: 8

In [0]:
# Asignamos dos particiones con un segundo parámetro
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 2)
rdd.getNumPartitions()

Out[2]: 2

In [0]:
rddE = sc.textFile("/FileStore/tables/empleados.txt")
rddE.getNumPartitions() 

Out[4]: 2

In [0]:
rddE = sc.textFile("/FileStore/tables/empleados.txt", 3)
rddE.getNumPartitions() 

Out[5]: 3

#### MapPartitions

In [0]:
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 2)

# La función recibe como parámetro un iterador con los elementos de la partición
def f(iterator): yield sum(iterator)
resultadoRdd = rdd.mapPartitions(f)
resultadoRdd.collect() 

Out[6]: [6, 15]

In [0]:
resultadoRdd2 = rdd.mapPartitions(lambda iterator: [list(iterator)])
resultadoRdd2.collect()

Out[7]: [[1, 1, 2, 2], [3, 3, 4, 5]]

#### mapPartitionsWithIndex

In [0]:
def mpwi(indice, iterador):
    return [(indice, list(iterador))]

resultadoRdd = rdd.mapPartitionsWithIndex(mpwi)
resultadoRdd.collect()

Out[8]: [(0, [1, 1, 2, 2]), (1, [3, 3, 4, 5])]

#### Modificar las particiones

In [0]:
# Ejemplo de coalesce
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 3)
rdd.getNumPartitions() 
rdd1p = rdd.coalesce(1)
rdd1p.getNumPartitions()

Out[9]: 1

In [0]:
# Ejemplo de repartition
rdd = sc.parallelize([1,1,2,2,3,3,4,5], 3)
rdd.getNumPartitions() 
rdd2p = rdd.repartition(2)
rdd2p.getNumPartitions()

Out[10]: 2